In [1]:
# Copyright 2023 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Each user is responsible for checking the content of datasets and the
# applicable licenses and determining if suitable for the intended use.

In [2]:
from IPython.display import display

def is_notebook():
    try:
        shell = get_ipython().__class__.__name__
        return shell in ('ZMQInteractiveShell',)  # Jupyter Notebook or JupyterLab
    except NameError:
        return False

<img src="https://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_models_01-getting-started/nvidia_logo.png" style="width: 90px; float: right;">

# Getting Started with Merlin Models: Develop a Model for MovieLens

This notebook is created using the latest stable [merlin-tensorflow](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/merlin/containers/merlin-tensorflow/tags) container. 

## Overview

[Merlin Models](https://github.com/NVIDIA-Merlin/models/) is a library for training recommender models. Merlin Models let Data Scientists and ML Engineers easily train standard RecSys models on their own dataset, getting GPU-accelerated models with best practices baked into the library. This will also let researchers to build custom models by incorporating standard components of deep learning recommender models, and then benchmark their new models on example offline datasets. Merlin Models is part of the [Merlin open source framework](https://developer.nvidia.com/nvidia-merlin).

Core features are:
- Many different recommender system architectures (tabular, two-tower, sequential) or tasks (binary, multi-class classification, multi-task)
- Flexible APIs targeted to both production and research
- Deep integration with NVIDIA Merlin platform, including NVTabular for ETL and Merlin Systems model serving


### Learning objectives

- Training [Facebook's DLRM model](https://arxiv.org/pdf/1906.00091.pdf) very easily with our high-level API.
- Understanding Merlin Models high-level API

## Downloading and preparing the dataset

In [3]:
import os
import merlin.models.tf as mm

from merlin.datasets.entertainment import get_movielens

2025-06-21 20:32:54.296504: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


/usr/local/lib/python3.10/dist-packages/merlin/dtypes/mappings/torch.py:43: UserWarning: PyTorch dtype mappings did not load successfully due to an error: No module named 'torch'
  warn(f"PyTorch dtype mappings did not load successfully due to an error: {exc.msg}")


[SOK INFO] Import /usr/local/lib/python3.10/dist-packages/merlin_sok-2.0.0-py3.10-linux-x86_64.egg/sparse_operation_kit/lib/libsparse_operation_kit.so
[SOK INFO] Initialize finished, communication tool: horovod


2025-06-21 20:32:58.632691: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1638] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1857 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650 SUPER, pci bus id: 0000:00:10.0, compute capability: 7.5
/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import tensorflow as tf
import json

We provide the `get_movielens()` function as a convenience to download the dataset, perform simple preprocessing, and split the data into training and validation datasets.

In [5]:
input_path = os.environ.get("INPUT_DATA_DIR", os.path.expanduser("~/merlin-models-data/movielens/"))
train, valid = get_movielens(variant="ml-1m", path=input_path)

## Training the DLRM Model with Merlin Models

We define the DLRM model, whose prediction task is a binary classification. From the `schema`, the categorical features are identified (and embedded) and the target columns are also automatically inferred, because of the schema tags. We talk more about the schema in the next [example notebook (02)](02-Merlin-Models-and-NVTabular-integration.ipynb),

In [6]:
if is_notebook():
    display(train.to_ddf().compute())
else:
    print(train.to_ddf().compute())

,title,gender,age,occupation,zipcode,TE_userId_rating,TE_age_rating,TE_gender_rating,TE_occupation_rating,TE_zipcode_rating,TE_movieId_rating,movieId,userId,genres,rating_binary,rating
0,1456,3,5,3,665,-0.649507,-1.108238,-0.559561,-0.763298,-0.770797,-0.504537,1460,442,"[4, 6]",1,4.0
1,222,3,3,7,254,0.687936,-0.500037,-0.551685,0.508136,0.768559,0.376705,222,514,"[5, 6]",1,4.0
2,229,4,3,13,718,0.741238,-0.564312,1.653488,1.356871,0.709963,0.610861,228,1383,"[3, 4]",1,5.0
3,645,3,6,10,208,0.866989,0.839907,-0.546343,-0.122714,0.826769,1.406878,644,455,"[19, 6]",1,4.0
4,581,4,3,10,397,-1.240364,-0.500037,1.802303,-0.095054,-1.361894,-1.483819,581,446,"[11, 7]",0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
800163,601,4,9,4,1279,0.090654,-0.517297,1.771640,-0.754857,0.085116,-0.344520,601,1325,"[3, 4]",1,4.0
800164,2402,4,3,9,562,-1.067486,-0.526585,1.771640,1.322863,-0.624986,-1.978015,2393,1034,[3],0,1.0
800165,1257,3,5,8,195,-1.282025,-1.080364,-0.598048,-1.383335,-1.321967,-1.271606,1258,472,"[5, 4, 18]",0,2.0
800166,714,3,4,4,1091,-1.062786,0.546378,-0.601528,-0.789640,-1.248707,0.421275,714,1040,[4],0,1.0


In [7]:
if is_notebook():
    display(valid.to_ddf().compute())
else:
    print(valid.to_ddf().compute())

,title,gender,age,occupation,zipcode,TE_userId_rating,TE_age_rating,TE_gender_rating,TE_occupation_rating,TE_zipcode_rating,TE_movieId_rating,movieId,userId,genres,rating_binary,rating
0,1714,3,4,8,616,0.126983,0.573534,-0.559561,-1.494169,0.127127,0.292199,1716,391,"[4, 8]",1,5.0
1,47,3,3,13,2772,0.067290,-0.500037,-0.551685,1.385532,0.058098,0.708996,47,4368,[3],1,4.0
2,2068,3,3,9,967,0.901214,-0.564312,-0.601528,1.192152,0.351155,0.719755,2068,1749,[3],1,4.0
3,109,3,4,14,440,0.248965,0.527548,-0.546343,1.421248,0.268185,0.522423,109,238,"[9, 12, 3, 17]",0,2.0
4,160,3,4,5,3173,0.520413,0.543985,-0.551685,0.329812,0.582084,1.013149,160,5379,"[4, 13]",1,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200036,80,3,8,14,1469,1.572813,2.757066,-0.551685,1.327085,1.799066,0.770341,80,1659,"[4, 7]",1,5.0
200037,2905,3,9,19,1541,0.102688,-0.464793,-0.551685,-2.739537,0.409004,-1.462613,2911,2321,[11],0,1.0
200038,256,4,4,13,98,1.121575,0.573534,1.771640,1.390369,1.130929,0.793883,256,563,"[3, 4, 8]",0,3.0
200039,578,3,5,7,161,-0.006593,-1.094344,-0.551685,0.508136,0.504562,-1.163358,578,3385,[3],0,3.0


In [8]:
# Ignoring the rating regression target column, to keep only the rating_binary target column for prediction
schema = train.schema.without(['rating'])

In [9]:
model = mm.DLRMModel(
    schema,
    embedding_dim=64,
    bottom_block=mm.MLPBlock([128, 64]),
    top_block=mm.MLPBlock([128, 64, 32]),
    prediction_tasks=mm.OutputBlock(schema),
)

In [10]:
k = 10
topk_metrics = [
    mm.RecallAt(k),
    mm.PrecisionAt(k),
    mm.NDCGAt(k),
    mm.MRRAt(k),
    mm.AvgPrecisionAt(k),
]

classification_metrics = [
    tf.keras.metrics.BinaryAccuracy(),
    tf.keras.metrics.Precision(),
    tf.keras.metrics.Recall(),
    tf.keras.metrics.AUC(),
]

# TODO: Read about this.
# Usage of the TopKMetricsAggregator helper:
# topk_agg = mm.TopKMetricsAggregator(topk_metrics)

# # New optimizaers api. You can use any of https://www.tensorflow.org/api_docs/python/tf/keras/optimizers#classes
# optimizer=tf.keras.optimizers.Adam(
#     learning_rate=0.001,
#     beta_1=0.9,
#     beta_2=0.999,
#     epsilon=1e-07,
#     amsgrad=False,
#     weight_decay=None,
#     clipnorm=None,
#     clipvalue=None,
#     global_clipnorm=None,
#     use_ema=False,
#     ema_momentum=0.99,
#     ema_overwrite_frequency=None
# )

# Legacy optimizaers api. You can use any of https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/legacy#classes
optimizer=tf.keras.optimizers.legacy.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    decay=0
)

model.compile(
    optimizer=optimizer,    
    # loss=tf.keras.losses.BinaryCrossentropy(), #  Default "loss" is "None". You can use any of https://www.tensorflow.org/api_docs/python/tf/keras/losses#classes
    metrics=topk_metrics + classification_metrics,
)

Next, we train the model.

In [11]:
print("--- Optimizer config ---")
optimizer_config = model.optimizer.get_config()
for key, value in optimizer_config.items():
    print(f"{key}: {value}")

--- Optimizer config ---
name: Adam
learning_rate: 0.001
decay: 0
beta_1: 0.9
beta_2: 0.999
epsilon: 1e-07
amsgrad: False


In [12]:
batch_size=1024
model.fit(train, batch_size=batch_size, epochs=1)

2025-06-21 20:33:02.993023: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


782/782 [==============================] - 20s 14ms/step - loss: 0.5424 - binary_accuracy: 0.7247 - precision: 0.7331 - recall: 0.8197 - auc: 0.7888 - recall_at_10: 0.9403 - precision_at_10: 0.9403 - ndcg_at_10: 0.9461 - mrr_at_10: 0.9808 - map_at_10: 0.9128 - regularization_loss: 0.0000e+00 - loss_batch: 0.5423


We evaluate the model...

In [13]:
metrics = model.evaluate(valid, batch_size=batch_size, return_dict=True)

2025-06-21 20:33:23.560189: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


196/196 [==============================] - 4s 10ms/step - loss: 0.5323 - binary_accuracy: 0.7301 - precision: 0.7603 - recall: 0.7757 - auc: 0.8018 - recall_at_10: 0.9845 - precision_at_10: 0.9845 - ndcg_at_10: 0.9875 - mrr_at_10: 0.9984 - map_at_10: 0.9782 - regularization_loss: 0.0000e+00 - loss_batch: 0.5324


... and check the evaluation metrics. As there are two columns tagged as target in the schema (`rating_binary` and `rating`), the model has two heads (multi-task learning), one for binary classification and the other for regression.  
You can see from the list below that default metrics are provided -- Precision, Recall, Accuracy and AUC for binary classification and `RMSE` for regression tasks. You can also provide your own metrics in `model.compile()`.

In [14]:
print(json.dumps(metrics, indent=2))

{
  "loss": 0.5322778224945068,
  "binary_accuracy": 0.7301403284072876,
  "precision": 0.7603118419647217,
  "recall": 0.7757427096366882,
  "auc": 0.8018380403518677,
  "recall_at_10": 0.98163241147995,
  "precision_at_10": 0.98163241147995,
  "ndcg_at_10": 0.9842684864997864,
  "mrr_at_10": 0.9974489808082581,
  "map_at_10": 0.9725860953330994,
  "regularization_loss": 0.0,
  "loss_batch": 0.5448323488235474
}


## Next steps

In the next example notebooks, we will show how the integration with NVTabular and how to explore different recommender models.